### Implementing FunkSVD - Solution

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 7. 10.  8.]
 [ 6. 10.  7.]
 [ 8.  9.  8.]
 [ 8. 10. 10.]
 [ 9.  9.  9.]
 [ 8.  9.  9.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 3 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [3]:
user_mat, movie_mat = FunkSVD(
    ratings_mat,
    latent_features=3, 
    learning_rate=0.005, 
    iters=10
)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 55.404810
2 		 41.503787
3 		 25.231384
4 		 11.782042
5 		 4.343802
6 		 1.529435
7 		 0.726526
8 		 0.526531
9 		 0.473939
10 		 0.455560


In [4]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 7.52110621  9.38393849  8.36920617]
 [ 7.13607795  8.70897824  7.48953845]
 [ 7.67179675  9.21095356  8.29099277]
 [ 8.51361546 10.28704775  9.14915943]
 [ 8.0583437   9.54205454  8.94245964]
 [ 7.60249318  9.29274232  8.79271083]]
[[ 7. 10.  8.]
 [ 6. 10.  7.]
 [ 8.  9.  8.]
 [ 8. 10. 10.]
 [ 9.  9.  9.]
 [ 8.  9.  9.]]


**The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations. Clearly the model is not done learning, but things are looking good.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 3 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 300.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about the `mean squared error` at the end of each training iteration?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, 
                              latent_features=3, 
                              learning_rate=0.005, 
                              iters=300)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 59.770290
2 		 47.309628
3 		 31.696231
4 		 16.902197
5 		 7.121745
6 		 2.629590
7 		 1.086039
8 		 0.634926
9 		 0.504284
10 		 0.460245
11 		 0.440601
12 		 0.428762
13 		 0.419830
14 		 0.412119
15 		 0.404951
16 		 0.398021
17 		 0.391182
18 		 0.384355
19 		 0.377499
20 		 0.370593
21 		 0.363625
22 		 0.356587
23 		 0.349479
24 		 0.342299
25 		 0.335052
26 		 0.327739
27 		 0.320367
28 		 0.312940
29 		 0.305466
30 		 0.297952
31 		 0.290406
32 		 0.282837
33 		 0.275254
34 		 0.267667
35 		 0.260086
36 		 0.252521
37 		 0.244984
38 		 0.237486
39 		 0.230036
40 		 0.222647
41 		 0.215328
42 		 0.208091
43 		 0.200947
44 		 0.193905
45 		 0.186974
46 		 0.180165
47 		 0.173486
48 		 0.166944
49 		 0.160547
50 		 0.154302
51 		 0.148215
52 		 0.142290
53 		 0.136533
54 		 0.130946
55 		 0.125532
56 		 0.120294
57 		 0.115233
58 		 0.110349
59 		 0.105641
60 		 0.101110
61 		 0.096754
62 		 0.092570
63 		 0.088556
64 

In [6]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 6.98801777  9.99293782  8.01501135]
 [ 6.00753073 10.0048037   6.9904128 ]
 [ 7.99536648  8.99641622  8.00667985]
 [ 8.00485454 10.00293168  9.99403779]
 [ 9.00254832  9.00206886  8.99630014]
 [ 7.9996749   8.9997555   9.0004557 ]]
[[ 7. 10.  8.]
 [ 6. 10.  7.]
 [ 8.  9.  8.]
 [ 8. 10. 10.]
 [ 9.  9.  9.]
 [ 8.  9.  9.]]


**In this case, we were able to completely reconstruct the item-movie matrix to closer to 0 mean squared error at the end of 300th iterations.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 3 latent features, a learning rate of 0.005, and 450 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [7]:
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10.,  8.],
        [ 6., 10.,  7.],
        [ 8.,  9.,  8.],
        [ 8., 10., 10.],
        [ 9.,  9.,  9.],
        [ 8.,  9.,  9.]])

In [8]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, 
                              latent_features=3, 
                              learning_rate=0.005, 
                              iters=450)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 62.826000
2 		 51.752629
3 		 37.028698
4 		 21.735639
5 		 10.288201
6 		 4.224337
7 		 1.815225
8 		 1.005938
9 		 0.740379
10 		 0.643648
11 		 0.601474
12 		 0.579182
13 		 0.565224
14 		 0.555195
15 		 0.547188
16 		 0.540290
17 		 0.534033
18 		 0.528165
19 		 0.522540
20 		 0.517074
21 		 0.511714
22 		 0.506423
23 		 0.501177
24 		 0.495959
25 		 0.490753
26 		 0.485548
27 		 0.480334
28 		 0.475103
29 		 0.469847
30 		 0.464559
31 		 0.459234
32 		 0.453867
33 		 0.448452
34 		 0.442985
35 		 0.437463
36 		 0.431882
37 		 0.426241
38 		 0.420538
39 		 0.414770
40 		 0.408938
41 		 0.403041
42 		 0.397080
43 		 0.391055
44 		 0.384969
45 		 0.378823
46 		 0.372620
47 		 0.366363
48 		 0.360057
49 		 0.353705
50 		 0.347313
51 		 0.340886
52 		 0.334431
53 		 0.327953
54 		 0.321459
55 		 0.314956
56 		 0.308453
57 		 0.301957
58 		 0.295476
59 		 0.289018
60 		 0.282592
61 		 0.276206
62 		 0.269870
63 		 0.263590
64

In [9]:
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is: {}".format(preds[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 7. Not bad!")

The predicted value for the missing rating is: 7.089435448741285

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 7. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 3 latent features, 0.005 learning rate, and 500 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [ ]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(
    first_1000_users, 
    latent_features=3, 
    learning_rate=0.005, 
    iters=500
)

`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [ ]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 14287.

The number of ratings made for user-movie pairs that didn't have ratings is 13835713


In [ ]:
# Test your results against the solution
assert num_ratings == 14287, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 13835713, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
